In [ ]:
from IPython.display import display, Math, Latex, HTML
HTML('''<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }
  
  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>''') 

In [ ]:
from data_exploration import *

In [ ]:
client, client_df = connect_to_influxdb()

In [ ]:
meeting_date="2019-01-10 14:00:00"
print("All database queries are set to select data before the meeting date: ",meeting_date)

In [ ]:
query_download = "SELECT * FROM SPEEDTEST_IPERF_DOWNLOAD WHERE PROVIDER!='iperf'AND DOWNLOAD>0 AND time <'"+meeting_date+"';"
download_df = get_dataframe_from_influxdb(client_df=client_df,query_influx=query_download,
                                          table_name='SPEEDTEST_IPERF_DOWNLOAD')

In [ ]:
query_upload = "SELECT * FROM SPEEDTEST_IPERF_UPLOAD WHERE PROVIDER!='iperf'AND UPLOAD>0 AND time <'"+meeting_date+"';"
upload_df = get_dataframe_from_influxdb(client_df=client_df,query_influx=query_upload,
                                          table_name='SPEEDTEST_IPERF_UPLOAD')

In [ ]:
query_ping = "SELECT * FROM SPEEDTEST_IPERF_PING WHERE PROVIDER!='iperf'AND PING>0 AND time <'"+meeting_date+"';"
ping_df = get_dataframe_from_influxdb(client_df=client_df,query_influx=query_ping,
                                          table_name='SPEEDTEST_IPERF_PING')
ping_df = ping_df[ping_df.PING != 1800000.000] ## Removed outier that was found in the ms sql table

# ConnectIN draft analysis

This PDF contains some initial observations of Internet speed test data being collected from various locations in Manitoba as part of the ConnectIN project. Note that these are initial observations and are still in draft format. 

At the time of the analysis in early January 2019, data had been collected from a total of 18 devices, primarily located in Manitoba. Device locations were not known yet at this time with the exception of device number 3, which is a control device set up in Cybera's offices in Calgary. 



# 1. Statistics by device

This data was collected from speedtest.net, which reports on download, upload and ping (latency) of each device. Speedtests are run every 3 hours and 42 minutes. On average, the data seen here corresponds to about 30 days of data. 

In [ ]:
device_numbers=get_tag_values_influxdb(client_influx=client,table_name='SPEEDTEST_IPERF_UPLOAD', tag_name='SK_PI')
device_numbers=list(map(int, device_numbers))
device_numbers= sorted(device_numbers)
print(device_numbers)

In [ ]:
#download_summary=mean_max_median_by1(download_df,'DOWNLOAD')
#download_line=go.Scatter(x=device_numbers,y=[50] * len(device_numbers), mode='markers',marker=dict(color='red'), name='50Mps')
#combined_bar_plot_3traces(xvalues=download_summary["SK_PI"],
#                         yvalues1=download_summary["max"],
#                         yvalues2=download_summary["mean"],
#                         yvalues3=download_summary["median"],
#                         name1="Max",
#                         name2="Mean",
#                         name3="Median",
#                         title="Download speed by device",
#                         ytitle="Mbps",
#                         line=download_line,
#                         stack=False)

In [ ]:
download_summary=mean_max_median_min_by1(download_df,'DOWNLOAD')
download_summary=download_summary[download_summary["SK_PI"]!=3]
device_numbers_d=download_summary["SK_PI"].unique()
download_line=go.Scatter(x=device_numbers_d,y=[50] * len(device_numbers_d), mode='markers',marker=dict(color='red'), name='50Mbps')
combined_bar_plot_4traces(xvalues=download_summary["SK_PI"],
                         yvalues1=download_summary["max"],
                         yvalues2=download_summary["mean"],
                         yvalues3=download_summary["median"],
                         yvalues4=download_summary["min"],
                         name1="Max",
                         name2="Mean",
                         name3="Median",
                         name4="Min",
                         title="Download speed by device",
                         ytitle="Mbps",
                         line=download_line,
                         stack=False)

In [ ]:
#upload_line=go.Scatter(x=device_numbers,y=[10] * len(device_numbers), mode='markers',marker=dict(color='red'), name='10Mps')
#upload_summary=mean_max_median_by1(upload_df,'UPLOAD')
#combined_bar_plot_3traces(xvalues=upload_summary["SK_PI"],
#                         yvalues1=upload_summary["max"],
#                         yvalues2=upload_summary["mean"],
#                         yvalues3=upload_summary["median"],
#                         name1="Max",
#                         name2="Mean",
#                         name3="Median",
#                         title="Upload speed ",
#                         ytitle="Mbps",
#                         line=upload_line,
 #                        stack=False)

In [ ]:
upload_summary=mean_max_median_min_by1(upload_df,'UPLOAD')
upload_summary=upload_summary[upload_summary["SK_PI"]!=3]

device_numbers_u=upload_summary["SK_PI"].unique()
upload_line=go.Scatter(x=device_numbers_u,y=[10] * len(device_numbers_u), mode='markers',marker=dict(color='red'), name='10Mbps')

combined_bar_plot_4traces(xvalues=upload_summary["SK_PI"],
                         yvalues1=upload_summary["max"],
                         yvalues2=upload_summary["mean"],
                         yvalues3=upload_summary["median"],
                         yvalues4=upload_summary["min"],
                         name1="Max",
                         name2="Mean",
                         name3="Median",
                         name4="Min",
                         title="Upload speed by device",
                         ytitle="Mbps",
                         line=upload_line,
                         stack=False)

In [ ]:
ping_summary=mean_max_median_by1(ping_df,'PING')
combined_bar_plot_3traces(xvalues=ping_summary["SK_PI"],
                         yvalues1=ping_summary["max"],
                         yvalues2=ping_summary["mean"],
                         yvalues3=ping_summary["median"],
                         name1="Max",
                         name2="Mean",
                         name3="Median",
                         title="Ping latency",
                         ytitle="Miliseconds",
                         stack=False)

## Summary

The plot of the download speeds shows that only devices 8 and 14 have averages or medians above the CRTC target of 50 Mbps and for upload speeds, devices 8, 9, 11, 14, 15, 17 have averages or medians above 10 Mbps. This is somewhat surprising, given that upload speeds are usually much worse. Given that these devices are all set up in professional buildings (e.g. schools, offices, etc), it is a bad sign that only few devices meet the CRTC targets. At the same time, one important caveat to remember is that we do not know how much bandwidth each organization is purchasing. For example, we would not expect to see speeds of 50 Mbps if only 10 Mbps are being purchased. 


### Brief note on box plots

Boxplots are an excellent way to summarize statistical data as they readily display how the data is distributed - giving you a better idea as to the range, and more importantly, the distinguishability of your data. However if you've never seen one before, here's a few important things to note about boxplots and what everything they display means. To begin, here is a labeled boxplot

![alt-text](https://cdn-images-1.medium.com/max/800/1*2c21SkzJMf3frPXPAR_gZA.png)

What we see here is a few important pieces 
1. The 'box' itself is centered on the median of the data, and has width equal to the interquartile range, which basically says that the box contains 50% of the data.
2. The lines that extend out are known as "whiskers" and extend from the bottom of the box, to a distance equal to 1.5 times the width of the box on each side (1.5 times the inter quartile range)
3. Finally, any points outside of the whiskers are drawn individually 

In [ ]:
simple_boxplot(dataframe=download_df,plot_value='DOWNLOAD',sort_value='SK_PI',
               title="Download speed", 
               ytitle="Mbps", downloadline=True)

In [ ]:
simple_boxplot(dataframe=upload_df,plot_value='UPLOAD',sort_value='SK_PI',
               title="Upload speed", 
               ytitle="Mbps", uploadline=True)

In [ ]:
simple_boxplot(dataframe=ping_df,plot_value='PING',sort_value='SK_PI',
               title="Ping latency", 
               ytitle="Miliseconds")

##  Summary

Looking at the box plot, device 14 shows a lot of variation in upload and download speeds, whereas many of the other devices show narrowly distributed data. Device 5 and 16 show a large number of outliers in regards to latency. Interestingly, devices displaying high bandwidth speeds show narrowly distributed latency and devices with low speeds have larger variability in latency. 

# 2. Statistics by Internet Service Provider

Each time a test is run, speedtest.net records which service provider the test originates from. At this time we are unclear about the exact methodology used by speedtest for determining the provider. 

In [ ]:
points_by_device=upload_df.groupby(['SK_PI']).size().reset_index(name='counts').sort_values('SK_PI')
providers_per_device = upload_df.groupby('SK_PI').apply(lambda x: x["PROVIDER"].unique()).apply(pd.Series)
providers_per_device['provider_count'] = providers_per_device.apply(lambda x: x.count(), axis=1)
provider_counts_per_device = pd.Series(providers_per_device['provider_count']).value_counts().reset_index()
points_by_device_by_provider=upload_df.groupby(['SK_PI', 'PROVIDER']).size().reset_index()

In [ ]:
combined_bar_plot_multitraces(dataframe=points_by_device_by_provider,
                              device_numbers=device_numbers,
                              points_by_device=points_by_device,
                              sort_value='PROVIDER',
                              ytitle="Percentage of data",
                              title="Percentage of service providers per device")

In [ ]:
upload_summary=mean_max_median_by1(upload_df,'UPLOAD',index_col='PROVIDER')
upload_line=go.Scatter(x=upload_summary["PROVIDER"].unique(),y=[10] * len(upload_summary["PROVIDER"].unique()), mode='markers',marker=dict(color='red'), name='10Mps')
combined_bar_plot_3traces(xvalues=upload_summary["PROVIDER"],
                         yvalues1=upload_summary["max"],
                         yvalues2=upload_summary["mean"],
                         yvalues3=upload_summary["median"],
                         name1="Max",
                         name2="Mean",
                         name3="Median",
                         title="Upload speed by provider",
                         ytitle="Mbps",
                         xtitle="Provider",
                         line=upload_line,
                         stack=False,
                         margin=True)

In [ ]:
download_summary=mean_max_median_by1(download_df,'DOWNLOAD',index_col='PROVIDER')
download_line=go.Scatter(x=upload_summary["PROVIDER"].unique(),y=[50] * len(upload_summary["PROVIDER"].unique()), mode='markers',marker=dict(color='red'), name='50Mps')
combined_bar_plot_3traces(xvalues=download_summary["PROVIDER"],
                         yvalues1=download_summary["max"],
                         yvalues2=download_summary["mean"],
                         yvalues3=download_summary["median"],
                         name1="Max",
                         name2="Mean",
                         name3="Median",
                         title="Download speed by provider",
                         ytitle="Mbps",
                         xtitle="Provider",
                         line=download_line,
                         stack=False,
                         margin=True)

In [ ]:
ping_summary=mean_max_median_by1(ping_df,'PING',index_col='PROVIDER')
combined_bar_plot_3traces(xvalues=ping_summary["PROVIDER"],
                         yvalues1=ping_summary["max"],
                         yvalues2=ping_summary["mean"],
                         yvalues3=ping_summary["median"],
                         name1="Max",
                         name2="Mean",
                         name3="Median",
                         title="Ping latency by provider",
                         ytitle="Miliseconds",
                         xtitle="Provider",
                         stack=False,
                         margin=True)

In [ ]:
simple_boxplot(dataframe=ping_df,plot_value='PING',sort_value='PROVIDER',
               title="Ping latency", 
               ytitle="Miliseconds")

## Summary:
Data has been collected on 10 internet service providers. Most of the devices have one provider, devices 8 and 14 have used 2 providers. This could be because the device was moved from one location to another or alternatively, it might reflect that certain devices utilize primary and secondary internet service providers. 

Bell and Keewaytinook were used by multiple devices, while the rest of the providers were only used by one device.  
   
For upload speeds, the CRTC target is 10Mbps: every provider except Bell MTS, High Speed Crow and  TeraGo Networks, the average uploadload speed is above 10.
   
For download speeds, the CRTC target is 50Mbps. Only Commstream Communications, MERLIN and Manitoba Hydro Internationl reached this number.

The largest ping latencies are Bell MTS, TeraGo Networks and Keewaytinook Okimakanak.

Based on conversation with the steering committee, there is some uncertainty about the accuracy of the service providers recorded here. This is something that will require further investigation going forward. 

# 3. Number of data points, device reporting times

Below we show a summary of the number of datapoints collected and over which period of time devices reported data. Two methods for collecting data are shown below: collectd and speedtest. The collectd tests report lesss data, but tests run far more frequently every 5 seconds, while speedtest reports on more data (upload, download speeds, and latency), but tests are run only every 3 hours and 42 minutes. 

**Commented out part of code will work only if collectd data is collected**

In [ ]:
#device_numbers=get_tag_values_influxdb(client_influx=client,table_name='PING', tag_name='SK_PI')
#device_numbers=list(map(int, device_numbers))
#device_numbers= sorted(device_numbers)
#print(device_numbers)

In [ ]:
#query_ping_counts = "SELECT COUNT(PING) FROM PING  WHERE time <'"+meeting_date+"'GROUP BY SK_PI;"
#ping_counts=get_stats_influxdb(client_influx=client,
#                               query_influx=query_ping_counts,
#                               stat_name='count',
#                               device_numbers=device_numbers)

In [ ]:
#query_ping_counts_time = "SELECT COUNT(PING) FROM PING WHERE time <'"+meeting_date+"' AND time >='"+meeting_date+"'-4w GROUP BY SK_PI ;"
#ping_counts_time = get_stats_influxdb(client_influx=client,
#                                      query_influx=query_ping_counts_time,
#                                      stat_name='count',
 #                                     device_numbers=device_numbers)

In [ ]:
#combined_bar_plot_2traces(xvalues=device_numbers,
#                          yvalues1=ping_counts_time,
#                          yvalues2=[a - b for a, b in zip(ping_counts, ping_counts_time)],
#                          name1='Last 4w',
#                          name2='The rest of the time',
#                          title="Comparing number of datapoints in last 4w vs entire time(collectd)",
#                          ytitle="Number of datapoints")

In [ ]:
#query_ping_last = "SELECT LAST(PING), time FROM PING WHERE time <'"+meeting_date+"' GROUP BY SK_PI;"
#result_ping_last=get_stats_influxdb(client_influx=client,
#                               query_influx=query_ping_last,
#                               stat_name='time',
#                               device_numbers=device_numbers)

In [ ]:
#query_ping_first = "SELECT FIRST(PING), time FROM PING WHERE time <'"+meeting_date+"' GROUP BY SK_PI;"
#result_ping_first=get_stats_influxdb(client_influx=client,
#                               query_influx=query_ping_first,
#                               stat_name='time',
#                               device_numbers=device_numbers)

In [ ]:
#print("Collectd reporting times:")
#data=[]
#for i in range(len(device_numbers)):
#    try:
#        result_ping_first[i] = dateutil.parser.parse(result_ping_first[i]).strftime('%Y-%m-%d %H:%M:%S')
#    except:
#        result_ping_first[i]=None
#    try:
#        result_ping_last[i] = dateutil.parser.parse(result_ping_last[i]).strftime('%Y-%m-%d %H:%M:%S')
#    except:
#        result_ping_last[i]=None
#    print("Device: ", device_numbers[i],"  was reporting from ", result_ping_first[i], " to ",result_ping_last[i])
#    trace = go.Scatter(x=[result_ping_first[i],result_ping_last[i]],y=[device_numbers[i],device_numbers[i]], 
#                       name = device_numbers[i],marker=dict(color=colors[i]))
#    data.append(trace)
#layout = dict(title = "Device reporting times(collectd)",xaxis=dict(title="Time"),
#        yaxis=dict(title="Device Number"))
#fig = go.Figure(data=data, layout=layout)
#iplot(fig)

In [ ]:
query_upload_counts = "SELECT COUNT(UPLOAD) FROM SPEEDTEST_IPERF_UPLOAD WHERE PROVIDER!='iperf' AND UPLOAD>0 AND time <'"+meeting_date+"' GROUP BY SK_PI;"
upload_counts=get_stats_influxdb(client_influx=client,
                               query_influx=query_upload_counts,
                               stat_name='count',
                               device_numbers=device_numbers)

In [ ]:
query_upload_counts_time = "SELECT COUNT(UPLOAD) FROM SPEEDTEST_IPERF_UPLOAD WHERE time <'"+meeting_date+"' AND time >='"+meeting_date+"'-4w AND PROVIDER!='iperf' AND UPLOAD>0 GROUP BY SK_PI ;"
upload_counts_time = get_stats_influxdb(client_influx=client,
                                      query_influx=query_upload_counts_time,
                                      stat_name='count',
                                      device_numbers=device_numbers)

In [ ]:
combined_bar_plot_2traces(xvalues=device_numbers,
                          yvalues1=upload_counts_time,
                          yvalues2=[a - b for a, b in zip(upload_counts, upload_counts_time)],
                          name1='Last 4w',
                          name2='The rest of the time',
                          title="Comparing number of datapoints in last 4w vs entire time(speedtest)",
                          ytitle="Number of datapoints")

In [ ]:
query_upload_last = "SELECT LAST(UPLOAD), time FROM SPEEDTEST_IPERF_UPLOAD WHERE PROVIDER!='iperf' AND UPLOAD>0 AND time <'"+meeting_date+"' GROUP BY SK_PI;"
result_upload_last=get_stats_influxdb(client_influx=client,
                               query_influx=query_upload_last,
                               stat_name='time',
                               device_numbers=device_numbers)

In [ ]:
query_upload_first = "SELECT FIRST(UPLOAD), time FROM SPEEDTEST_IPERF_UPLOAD WHERE PROVIDER!='iperf' AND UPLOAD>0 AND time <'"+meeting_date+"' GROUP BY SK_PI;"
result_upload_first=get_stats_influxdb(client_influx=client,
                               query_influx=query_upload_first,
                               stat_name='time',
                               device_numbers=device_numbers)

In [ ]:
print("Speedtest reporting times:")
data=[]
for i in range(len(device_numbers)):
    try:
        result_upload_first[i] = dateutil.parser.parse(result_upload_first[i]).strftime('%Y-%m-%d %H:%M:%S')
    except:
        result_upload_first[i]=None
    try:    
        result_upload_last[i] = dateutil.parser.parse(result_upload_last[i]).strftime('%Y-%m-%d %H:%M:%S')
    except:
        result_upload_last[i]=None
    print("Device: ", device_numbers[i],"  was reporting from ", result_upload_first[i], " to ",result_upload_last[i])
    trace = go.Scatter(x=[result_upload_first[i],result_upload_last[i]],y=[device_numbers[i],device_numbers[i]], 
                       name = device_numbers[i],marker=dict(color=colors[i]))
    data.append(trace)
layout = dict(title = "Device reporting times(speedtest)",xaxis=dict(title="Time"),
        yaxis=dict(title="Device Number"))
fig = go.Figure(data=data, layout=layout)
iplot(fig)

## Summary: 

There is at least eom data from 18 devices. Devices 1, 2, 4, 6, 8 started reporting speedtest data and then stopped. Device 13 has not sent any speedtest data (sent several collectd data points) and devices 10, 13, 14 are not sending collectd data but sending speedtest data.    

More detailed timeline can be found in [grafana](https://grafana-connectin.cybera.ca) (login required).

The devices that are no longer reporting data should be followed-up with and reset. It appears that the speedtest may be running more reliably, as connectd tests have shown a higher failure rate than speedtest tests. 